In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("Retail Sales Analysis") \
.getOrCreate()

sales_data = [
("T001","North","Delhi","Store-01","Laptop","2024-01-01",75000),
("T002","North","Delhi","Store-01","Mobile","2024-01-02",32000),
("T003","North","Chandigarh","Store-02","Tablet","2024-01-03",26000),
("T004","South","Bangalore","Store-03","Laptop","2024-01-01",78000),
("T005","South","Chennai","Store-04","Mobile","2024-01-02",30000),
("T006","South","Bangalore","Store-03","Tablet","2024-01-03",24000),
("T007","East","Kolkata","Store-05","Laptop","2024-01-01",72000),
("T008","East","Kolkata","Store-05","Mobile","2024-01-02",28000),
("T009","East","Patna","Store-06","Tablet","2024-01-03",23000),
("T010","West","Mumbai","Store-07","Laptop","2024-01-01",80000),
("T011","West","Mumbai","Store-07","Mobile","2024-01-02",35000),
("T012","West","Pune","Store-08","Tablet","2024-01-03",27000),
("T013","North","Delhi","Store-01","Laptop","2024-01-04",76000),
("T014","South","Chennai","Store-04","Laptop","2024-01-04",79000),
("T015","East","Patna","Store-06","Mobile","2024-01-04",29000),
("T016","West","Pune","Store-08","Laptop","2024-01-04",77000),
("T017","North","Chandigarh","Store-02","Mobile","2024-01-05",31000),
("T018","South","Bangalore","Store-03","Mobile","2024-01-05",34000),
("T019","East","Kolkata","Store-05","Tablet","2024-01-05",25000),
("T020","West","Mumbai","Store-07","Tablet","2024-01-05",29000),
("T021","North","Delhi","Store-01","Tablet","2024-01-06",28000),
("T022","South","Chennai","Store-04","Tablet","2024-01-06",26000),
("T023","East","Patna","Store-06","Laptop","2024-01-06",74000),
("T024","West","Pune","Store-08","Mobile","2024-01-06",33000)
]
columns = [
"txn_id","region","city","store_id",
"product","sale_date","amount"
]
df_sales = spark.createDataFrame(sales_data, columns)
df_sales.show(5)
df_sales.printSchema()

#Exercise - 1
1. Select only txn_id , region , product , and amount
2. Rename amount to revenue
3. Create a derived column amount_in_thousands
4. Select distinct combinations of region and product
5. Select all columns but exclude store_id
6. Create a new column sale_year extracted from sale_date
7. Reorder columns in a business-friendly format

In [ ]:
df_select = df_sales.select("txn_id","region","product","amount")
df_select.show(5)


In [ ]:
df_renamed = df_sales.select(
    "txn_id","region","product",
    df_sales["amount"].alias("revenue")
)
df_renamed.show(5)


In [ ]:
from pyspark.sql.functions import round, col
df_amt_k = df_sales.withColumn(
    "amount_in_thousands",
    round(col("amount") / 1000.0, 2)
).select("txn_id", "region", "product", "amount", "amount_in_thousands")
df_amt_k.show(5)



In [ ]:

df_distinct = df_sales.select("region", "product").distinct()
df_distinct.show()

In [ ]:

df_no_store = df_sales.drop("store_id")
df_no_store.show(5, truncate=False)


In [ ]:
from pyspark.sql.functions import year, to_date
df_with_year = (
    df_sales
    .withColumn("sale_date", to_date(col("sale_date")))  # cast to date
    .withColumn("sale_year", year(col("sale_date")))
    .select("txn_id", "region", "city", "store_id", "product", "sale_date", "sale_year", "amount")
)

df_with_year.show(5, truncate=False)


In [12]:

df_with_year = (
    df_sales
    .withColumn("sale_date", to_date(col("sale_date")))  # cast to date
    .withColumn("sale_year", year(col("sale_date")))
    .select("txn_id", "region", "city", "store_id", "product", "sale_date", "sale_year", "amount")
)

df_with_year.show(5, truncate=False)


+------+------+----------+--------+-------+----------+---------+------+
|txn_id|region|city      |store_id|product|sale_date |sale_year|amount|
+------+------+----------+--------+-------+----------+---------+------+
|T001  |North |Delhi     |Store-01|Laptop |2024-01-01|2024     |75000 |
|T002  |North |Delhi     |Store-01|Mobile |2024-01-02|2024     |32000 |
|T003  |North |Chandigarh|Store-02|Tablet |2024-01-03|2024     |26000 |
|T004  |South |Bangalore |Store-03|Laptop |2024-01-01|2024     |78000 |
|T005  |South |Chennai   |Store-04|Mobile |2024-01-02|2024     |30000 |
+------+------+----------+--------+-------+----------+---------+------+
only showing top 5 rows


#EXERCISE SET 2 — FILTER OPERATIONS

Objective
Understand row-level filtering and predicate pushdown.
Exercises
1. Filter transactions where amount > 50000
2. Filter only Laptop sales
3. Filter sales from North and South regions
4. Filter sales between 25000 and 75000
5. Filter transactions from Delhi stores only
6. Apply multiple filters using both filter and where
7. Change the order of filters and compare explain(True)
8. Identify which filters Spark pushes down

In [ ]:
df_filter1 = df_sales.filter(df_sales["amount"] > 50000)
df_filter1.show(5)

In [ ]:
df_filter2 = df_sales.filter(df_sales["product"] == "Laptop")
df_filter2.show()

In [ ]:
df_filter3 = df_sales.filter(
    (df_sales["region"] == "North") |
    (df_sales["region"] == "South")
)
df_filter3.show()


In [ ]:
df_filter4 = df_sales.filter(
    (df_sales["amount"] >= 25000) &
    (df_sales["amount"] <= 75000)
)
df_filter4.show()

In [ ]:
df_filter5 = df_sales.filter(df_sales["city"] == "Delhi")
df_filter5.show()

In [ ]:

df_multi_chain = (
    df_sales
    .filter(col("region") == "North")
    .where(col("product") == "Laptop")
    .filter(col("amount") > 50000)
)
df_multi_chain.show(10, truncate=False)

In [ ]:

plan_a = (
    df_sales
    .filter(col("region") == "North")
    .filter(col("product") == "Laptop")
    .filter(col("amount") > 50000)
)
plan_b = (
    df_sales
    .filter(col("amount") > 50000)
    .filter(col("product") == "Laptop")
    .filter(col("region") == "North")
)

plan_a.explain(True)
plan_b.explain(True)


#EXERCISE SET 3 — GROUPBY & AGGREGATE
FUNCTIONS

Objective
Perform summarization and understand shuffles.
Exercises
1. Total sales amount per region
2. Average sales amount per product
3. Maximum sale per city
4. Minimum sale per store
5. Count of transactions per region
6. Total revenue per store
7. Region-wise product sales count
8. Average transaction value per city
9. Identify regions with total sales above a threshold
10. Use explain(True) and identify shuffle stages

In [ ]:
from pyspark.sql.functions import col, sum as _sum, avg, max as _max, min as _min, count
total_sales_per_region = df_sales.groupBy("region").agg(_sum("amount").alias("total_amount"))
total_sales_per_region.orderBy(col("total_amount").desc()).show(truncate=False)

In [ ]:
avg_sales_per_product = df_sales.groupBy("product").agg(avg("amount").alias("avg_amount"))
avg_sales_per_product.orderBy(col("avg_amount").desc()).show()

In [ ]:
max_sale_per_city = df_sales.groupBy("city").agg(_max("amount").alias("max_amount"))
max_sale_per_city.show()

In [ ]:
min_sale_per_store = df_sales.groupBy("store_id").agg(_min("amount").alias("min_amount"))
min_sale_per_store.show()

In [ ]:
count_transactions_per_region = df_sales.groupBy("region").agg(count("*").alias("transaction_count"))
count_transactions_per_region.show()